In [115]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate, BatchNormalization, Dropout, LeakyReLU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers.legacy import Adam, RMSprop

from table_evaluator import load_data, TableEvaluator
#from sdv.evaluation import evaluate


In [5]:
df_parkinsons = pd.read_csv("parkinsons_updrs.data.csv")
display(df_parkinsons)

,index,subject#,age,sex,test_time,motor_UPDRS,total_UPDRS,Jitter(%),Jitter(Abs),Jitter:RAP,...,Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,Shimmer:DDA,NHR,HNR,RPDE,DFA,PPE
0,0,1,72,0,5.6431,28.199,34.398,0.00662,0.000034,0.00401,...,0.230,0.01438,0.01309,0.01662,0.04314,0.014290,21.640,0.41888,0.54842,0.16006
1,1,1,72,0,12.6660,28.447,34.894,0.00300,0.000017,0.00132,...,0.179,0.00994,0.01072,0.01689,0.02982,0.011112,27.183,0.43493,0.56477,0.10810
2,2,1,72,0,19.6810,28.695,35.389,0.00481,0.000025,0.00205,...,0.181,0.00734,0.00844,0.01458,0.02202,0.020220,23.047,0.46222,0.54405,0.21014
3,3,1,72,0,25.6470,28.905,35.810,0.00528,0.000027,0.00191,...,0.327,0.01106,0.01265,0.01963,0.03317,0.027837,24.445,0.48730,0.57794,0.33277
4,4,1,72,0,33.6420,29.187,36.375,0.00335,0.000020,0.00093,...,0.176,0.00679,0.00929,0.01819,0.02036,0.011625,26.126,0.47188,0.56122,0.19361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,5870,42,61,0,142.7900,22.485,33.485,0.00406,0.000031,0.00167,...,0.160,0.00973,0.01133,0.01549,0.02920,0.025137,22.369,0.64215,0.55314,0.21367
5871,5871,42,61,0,149.8400,21.988,32.988,0.00297,0.000025,0.00119,...,0.215,0.01052,0.01277,0.01904,0.03157,0.011927,22.886,0.52598,0.56518,0.12621
5872,5872,42,61,0,156.8200,21.495,32.495,0.00349,0.000025,0.00152,...,0.244,0.01371,0.01456,0.01877,0.04112,0.017701,25.065,0.47792,0.57888,0.14157
5873,5873,42,61,0,163.7300,21.007,32.007,0.00281,0.000020,0.00128,...,0.131,0.00693,0.00870,0.01307,0.02078,0.007984,24.422,0.56865,0.56327,0.14204


In [6]:
df_parkinsons.columns

Index(['index', 'subject#', 'age', 'sex', 'test_time', 'motor_UPDRS',
       'total_UPDRS', 'Jitter(%)', 'Jitter(Abs)', 'Jitter:RAP', 'Jitter:PPQ5',
       'Jitter:DDP', 'Shimmer', 'Shimmer(dB)', 'Shimmer:APQ3', 'Shimmer:APQ5',
       'Shimmer:APQ11', 'Shimmer:DDA', 'NHR', 'HNR', 'RPDE', 'DFA', 'PPE'],
      dtype='object')

In [7]:
df_parkinsons.describe()

,index,subject#,age,sex,test_time,motor_UPDRS,total_UPDRS,Jitter(%),Jitter(Abs),Jitter:RAP,...,Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,Shimmer:DDA,NHR,HNR,RPDE,DFA,PPE
count,5875.000000,5875.000000,5875.000000,5875.000000,5875.000000,5875.000000,5875.000000,5875.000000,5875.000000,5875.000000,...,5875.000000,5875.000000,5875.000000,5875.000000,5875.000000,5875.000000,5875.000000,5875.000000,5875.000000,5875.000000
mean,2937.000000,21.494128,64.804936,0.317787,92.863722,21.296229,29.018942,0.006154,0.000044,0.002987,...,0.310960,0.017156,0.020144,0.027481,0.051467,0.032120,21.679495,0.541473,0.653240,0.219589
std,1696.110747,12.372279,8.821524,0.465656,53.445602,8.129282,10.700283,0.005624,0.000036,0.003124,...,0.230254,0.013237,0.016664,0.019986,0.039711,0.059692,4.291096,0.100986,0.070902,0.091498
min,0.000000,1.000000,36.000000,0.000000,-4.262500,5.037700,7.000000,0.000830,0.000002,0.000330,...,0.026000,0.001610,0.001940,0.002490,0.004840,0.000286,1.659000,0.151020,0.514040,0.021983
25%,1468.500000,10.000000,58.000000,0.000000,46.847500,15.000000,21.371000,0.003580,0.000022,0.001580,...,0.175000,0.009280,0.010790,0.015665,0.027830,0.010955,19.406000,0.469785,0.596180,0.156340
50%,2937.000000,22.000000,65.000000,0.000000,91.523000,20.871000,27.576000,0.004900,0.000035,0.002250,...,0.253000,0.013700,0.015940,0.022710,0.041110,0.018448,21.920000,0.542250,0.643600,0.205500
75%,4405.500000,33.000000,72.000000,1.000000,138.445000,27.596500,36.399000,0.006800,0.000053,0.003290,...,0.365000,0.020575,0.023755,0.032715,0.061735,0.031463,24.444000,0.614045,0.711335,0.264490
max,5874.000000,42.000000,85.000000,1.000000,215.490000,39.511000,54.992000,0.099990,0.000446,0.057540,...,2.107000,0.162670,0.167020,0.275460,0.488020,0.748260,37.875000,0.966080,0.865600,0.731730


Training a base model

In [147]:
df_train = df_parkinsons.copy()
df_train = df_train.drop(['index', 'subject#'], axis=1)

In [148]:
X = df_train.copy()
X = X.drop('total_UPDRS', axis=1)
y = df_train['total_UPDRS']

In [149]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=48)

In [150]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [151]:
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train_scaled, y_train)
predictions = rf_model.predict(X_test_scaled)
mae = metrics.mean_absolute_error(y_test, predictions)
print(f'Mean Absolute Error: {mae}')

Mean Absolute Error: 0.2104556212765951


Generating new data using GAN

In [90]:
#CONSTANTS
np.random.seed(42)
num_samples = 1000
scaler = MinMaxScaler()
data_scaled = df_parkinsons.copy()
data_scaled = data_scaled.drop(['index', 'subject#'], axis=1)
data_scaled = scaler.fit_transform(data_scaled)
num_features = data_scaled.shape[1]
real_data = data_scaled.copy()
latent_dim = 5
output_dim = num_features
input_dim = num_features
epochs = 5000
batch_size = 128


In [91]:
#Generator
def build_generator(latent_dim, output_dim):
    model = Sequential()
    model.add(Dense(512, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization())
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization())
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization())
    model.add(Dense(output_dim, activation='tanh'))
    
    return model


In [92]:
# Discriminator
def build_discriminator(input_dim):
    model = Sequential()
    model.add(Dense(512, input_dim=input_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    
    return model

In [93]:
# GAN
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

In [94]:
discriminator = build_discriminator(input_dim)
discriminator.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0002, beta_1=0.5))

generator = build_generator(latent_dim, output_dim)
generator.compile(loss='mae', optimizer=Adam(learning_rate=0.0002))


discriminator.trainable = False
gan = build_gan(generator, discriminator)
gan.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0002, beta_1=0.5))


/Users/daksh/Desktop/MDSA/Generative AI/.venv/lib/python3.11/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [95]:
print(real_data.shape)

(5875, 21)


In [96]:
#Train GAN
for epoch in range(epochs + 1):
    # Generate random noise as input to the generator
    noise = np.random.normal(0, 1, size=(batch_size, latent_dim))

    # Generate synthetic data with the generator
    generated_data = generator.predict(noise)

    # Create a dataset with real and generated data
    batch_index_real = np.random.randint(0, real_data.shape[0], batch_size)
    X_real = real_data[batch_index_real]
    X_fake = generated_data

    # Labels for the discriminator
    y_real = np.ones((batch_size, 1))
    y_fake = np.zeros((batch_size, 1))

    # Train the discriminator
    d_loss_real = discriminator.train_on_batch(X_real, y_real)
    d_loss_fake = discriminator.train_on_batch(X_fake, y_fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Generate noise for the generator input
    noise = np.random.normal(0, 1, size=(batch_size, latent_dim))

    # Labels for the generator
    y_gen = np.ones((batch_size, 1))

    # Train the generator
    g_loss = gan.train_on_batch(noise, y_gen)

    # Print progress
    if epoch % 500 == 0:
        print(f"Epoch {epoch}, D Loss: {d_loss}, G Loss: {g_loss}")


4/4 [==============================] - 0s 1ms/step
Epoch 0, D Loss: 0.2498294934630394, G Loss: 0.20666658878326416
4/4 [==============================] - 0s 1ms/step
Epoch 500, D Loss: 0.11974457651376724, G Loss: 0.5706472396850586
4/4 [==============================] - 0s 1ms/step
Epoch 1000, D Loss: 0.04995597992092371, G Loss: 0.8281770944595337
4/4 [==============================] - 0s 1ms/step
Epoch 1500, D Loss: 0.04339982848614454, G Loss: 0.8968175053596497
4/4 [==============================] - 0s 1ms/step
Epoch 2000, D Loss: 0.025933212600648403, G Loss: 0.926368236541748
4/4 [==============================] - 0s 1ms/step
Epoch 2500, D Loss: 0.012131423456594348, G Loss: 0.9961717128753662
4/4 [==============================] - 0s 2ms/step
Epoch 3000, D Loss: 0.012879916466772556, G Loss: 0.9713548421859741
4/4 [==============================] - 0s 2ms/step
Epoch 3500, D Loss: 0.009652781765908003, G Loss: 0.9642804861068726
4/4 [==============================] - 0s 1ms/ste

In [97]:
def generate_data(generator, latent_dim, num_samples):
    noise = np.random.normal(0, 1, (num_samples, latent_dim))
    generated_data = generator.predict(noise)

    return generated_data


In [98]:
# Assuming 'generator' is your trained generator model
latent_dim = 5  # Adjust based on your model's latent dimension
num_generated_samples = len(real_data)  # Generate the same number of samples as real_data

# Generate synthetic data
generated_data = generate_data(generator, latent_dim, num_generated_samples)

display(generated_data)

184/184 [==============================] - 0s 1ms/step


array([[ 0.47941434,  0.99277943,  0.47805488, ...,  0.38876855,
         0.06009236,  0.48384523],
       [ 0.6616173 ,  0.81293505,  0.36089408, ...,  0.3254897 ,
         0.3163513 ,  0.3206823 ],
       [ 0.45081913,  0.99955374,  0.83325195, ...,  0.28682438,
         0.31005934,  0.34384194],
       ...,
       [ 0.6331846 , -0.02589866,  0.3660676 , ...,  0.39283773,
         0.5375901 ,  0.45089734],
       [ 0.48209924,  0.99645025,  0.9487085 , ...,  0.03831949,
         0.56711   ,  0.33265778],
       [ 0.8327544 ,  0.99981725,  0.8543025 , ...,  0.78469884,
         0.5891455 ,  0.6006068 ]], dtype=float32)

In [99]:
original_data = df_parkinsons.copy()
original_data = original_data.drop(['index', 'subject#'], axis=1)
original_data[original_data.columns] = scaler.fit_transform(original_data[original_data.columns])


In [100]:
generated_df = pd.DataFrame(generated_data, columns=original_data.columns)
#from table_evaluator import load_data, TableEvaluator
#table_evaluator = TableEvaluator(generated_df, original_data)
#table_evaluator.visual_evaluation()

In [101]:
#print(generated_df.columns)
#print(original_data.columns)
#evaluate(generated_df, original_data)

In [106]:
from sklearn.metrics import mean_squared_error

# Assuming 'real_data' and 'generated_data' are your real and generated datasets

# Calculate Mean Squared Error
mse = mean_squared_error(original_data, generated_df)

# Calculate Maximum Possible MSE (assuming max possible error is the variance of the real data)
max_possible_mse = np.var(real_data)

# Calculate Percentage Similarity
percentage_similarity = max(0, 100 * (1 - mse / max_possible_mse))

# Display the Percentage Similarity
print("Percentage Similarity:", percentage_similarity)


Percentage Similarity: 0
Generated Samples:


Training wGAN

In [116]:
import numpy as np
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop

# Constants
NOISE_DIM = 100
NUM_FEATURES = data_scaled.shape[1]
BATCH_SIZE = 64
NEW_TRAINING_STEPS = 5000

# Generator
def build_generator(latent_dim, output_dim):
    model = Sequential()
    model.add(Dense(512, input_dim=latent_dim, activation='relu'))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(output_dim, activation='tanh'))
    return model

# Discriminator
def build_critic(input_dim):
    model = Sequential()
    model.add(Dense(512, input_dim=input_dim, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(1))  # No activation in the last layer for WGAN
    return model

# Compile the critic with RMSprop optimizer
def compile_critic(critic):
    optimizer = RMSprop(lr=0.00005)
    critic.compile(loss='mean_squared_error', optimizer=optimizer)

# Compile the generator with RMSprop optimizer
def compile_generator(generator):
    optimizer = RMSprop(lr=0.00005)
    generator.compile(loss='mean_squared_error', optimizer=optimizer)

# Build the Wasserstein GAN
def build_wgan(generator, critic):
    critic.trainable = False  # Freeze the critic during generator training

    model = Sequential()
    model.add(generator)
    model.add(critic)
    return model

# Compile the Wasserstein GAN with RMSprop optimizer
def compile_wgan(wgan):
    optimizer = RMSprop(lr=0.00005)
    wgan.compile(loss='mean_squared_error', optimizer=optimizer)


# Build the Wasserstein GAN
generator = build_generator(NOISE_DIM, NUM_FEATURES)
critic = build_critic(NUM_FEATURES)
wgan = build_wgan(generator, critic)

# Compile the generator, critic, and wgan
compile_generator(generator)
compile_critic(critic)
compile_wgan(wgan)

# Train the Wasserstein GAN
for step in range(NEW_TRAINING_STEPS):
    noise = np.random.normal(0, 1, size=(BATCH_SIZE, NOISE_DIM))
    generated_batch = generator.predict(noise)

    idx = np.random.randint(0, real_data.shape[0], BATCH_SIZE)
    batch_index_real = np.random.randint(0, real_data.shape[0], BATCH_SIZE)

    real_batch = real_data[batch_index_real]

    # Train critic
    critic_loss_real = critic.train_on_batch(real_batch, np.ones((BATCH_SIZE, 1)))
    critic_loss_fake = critic.train_on_batch(generated_batch, -np.ones((BATCH_SIZE, 1)))
    critic_loss = 0.5 * np.add(critic_loss_real, critic_loss_fake)

    # Train generator
    generator_loss = wgan.train_on_batch(noise, np.ones((BATCH_SIZE, 1)))

    if step % 500 == 0:
        print(f"Step: {step}, Critic Loss: {critic_loss}, Generator Loss: {generator_loss}")


2/2 [==============================] - 0s 2ms/step
Step: 0, Critic Loss: 1.0497429370880127, Generator Loss: 1.0892271995544434
2/2 [==============================] - 0s 2ms/step
Step: 500, Critic Loss: 1.8398211002349854, Generator Loss: 0.1825764924287796
2/2 [==============================] - 0s 2ms/step
Step: 1000, Critic Loss: 1.8381247520446777, Generator Loss: 0.1822138875722885
2/2 [==============================] - 0s 2ms/step
Step: 1500, Critic Loss: 1.8322136402130127, Generator Loss: 0.1818707287311554
2/2 [==============================] - 0s 2ms/step
Step: 2000, Critic Loss: 1.833679437637329, Generator Loss: 0.1818680465221405
2/2 [==============================] - 0s 2ms/step
Step: 2500, Critic Loss: 1.826719045639038, Generator Loss: 0.18177416920661926
2/2 [==============================] - 0s 2ms/step
Step: 3000, Critic Loss: 1.8379429578781128, Generator Loss: 0.18181169033050537
2/2 [==============================] - 0s 1ms/step
Step: 3500, Critic Loss: 1.836800336

In [117]:
def generate_synthetic_data(generator, num_samples=1000, noise_dim=100):
    noise = np.random.normal(0, 1, (num_samples, noise_dim))
    synthetic_data = generator.predict(noise)

    return synthetic_data


In [118]:
original_data = df_parkinsons.copy()
original_data = original_data.drop(['index', 'subject#'], axis=1)
original_data[original_data.columns] = scaler.fit_transform(original_data[original_data.columns])

In [126]:
num_generated_samples = len(real_data)  
synthetic_data = generate_synthetic_data(generator, num_samples=num_generated_samples)
synthetic_df = pd.DataFrame(synthetic_data, columns=original_data.columns)

184/184 [==============================] - 0s 1ms/step


,age,sex,test_time,motor_UPDRS,total_UPDRS,Jitter(%),Jitter(Abs),Jitter:RAP,Jitter:PPQ5,Jitter:DDP,...,Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,Shimmer:DDA,NHR,HNR,RPDE,DFA,PPE
0,-0.068899,0.999800,-0.999900,-0.999983,-0.999978,0.999995,-0.342527,-0.890447,0.999993,-0.921212,...,0.977395,-0.805301,-0.999988,0.999995,-0.999926,0.973918,0.999990,-0.999979,-0.999988,0.999985
1,-0.071381,0.998782,-0.999387,-0.999896,-0.999822,0.999954,-0.332858,-0.859205,0.999926,-0.896695,...,0.953028,-0.778823,-0.999887,0.999955,-0.999593,0.949310,0.999915,-0.999867,-0.999914,0.999895
2,-0.080529,0.999457,-0.999759,-0.999961,-0.999949,0.999984,-0.336686,-0.851514,0.999983,-0.923678,...,0.966153,-0.807375,-0.999960,0.999984,-0.999800,0.962462,0.999972,-0.999939,-0.999967,0.999955
3,-0.084340,0.999236,-0.999654,-0.999947,-0.999893,0.999975,-0.343285,-0.862507,0.999968,-0.902821,...,0.962205,-0.786541,-0.999947,0.999980,-0.999763,0.958663,0.999948,-0.999921,-0.999949,0.999943
4,-0.066972,0.999347,-0.999662,-0.999945,-0.999906,0.999977,-0.337691,-0.866303,0.999968,-0.904738,...,0.962938,-0.794513,-0.999959,0.999982,-0.999759,0.955450,0.999959,-0.999929,-0.999949,0.999939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,-0.070225,0.998627,-0.999386,-0.999887,-0.999763,0.999948,-0.343441,-0.867459,0.999923,-0.906301,...,0.953766,-0.788657,-0.999898,0.999955,-0.999548,0.945257,0.999916,-0.999859,-0.999902,0.999884
5871,-0.076075,0.999411,-0.999785,-0.999954,-0.999925,0.999983,-0.316984,-0.866293,0.999979,-0.901103,...,0.967852,-0.789033,-0.999960,0.999981,-0.999775,0.958652,0.999969,-0.999942,-0.999964,0.999953
5872,-0.063269,0.998745,-0.999303,-0.999877,-0.999798,0.999950,-0.335857,-0.859194,0.999948,-0.902047,...,0.953736,-0.787784,-0.999889,0.999952,-0.999585,0.942086,0.999901,-0.999855,-0.999883,0.999873
5873,-0.070244,0.999161,-0.999574,-0.999922,-0.999879,0.999969,-0.334943,-0.853228,0.999955,-0.907949,...,0.957686,-0.789605,-0.999929,0.999969,-0.999727,0.953501,0.999935,-0.999882,-0.999941,0.999918


,age,sex,test_time,motor_UPDRS,total_UPDRS,Jitter(%),Jitter(Abs),Jitter:RAP,Jitter:PPQ5,Jitter:DDP,...,Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,Shimmer:DDA,NHR,HNR,RPDE,DFA,PPE
0,0.734694,0.0,0.045076,0.671862,0.570887,0.058390,0.071164,0.064324,0.039635,0.064433,...,0.098030,0.079287,0.067543,0.051764,0.079267,0.018723,0.551717,0.328638,0.097793,0.194544
1,0.734694,0.0,0.077034,0.679056,0.581222,0.021884,0.032819,0.017305,0.015478,0.017303,...,0.073522,0.051720,0.053186,0.052753,0.051699,0.014474,0.704771,0.348330,0.144300,0.121335
2,0.734694,0.0,0.108957,0.686250,0.591536,0.040137,0.050458,0.030065,0.023868,0.030178,...,0.074483,0.035577,0.039375,0.044291,0.035556,0.026651,0.590568,0.381812,0.085362,0.265104
3,0.734694,0.0,0.136105,0.692342,0.600308,0.044877,0.054856,0.027618,0.031969,0.027673,...,0.144642,0.058674,0.064878,0.062791,0.058632,0.036834,0.629169,0.412583,0.181761,0.437884
4,0.734694,0.0,0.172487,0.700522,0.612081,0.025413,0.040353,0.010488,0.012585,0.010486,...,0.072081,0.032162,0.044524,0.057515,0.032121,0.015160,0.675585,0.393664,0.134202,0.241814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5870,0.510204,0.0,0.669173,0.506111,0.551863,0.032574,0.065142,0.023422,0.018082,0.023420,...,0.064392,0.050416,0.056882,0.047624,0.050416,0.033224,0.571847,0.602569,0.111219,0.270078
5871,0.510204,0.0,0.701255,0.491694,0.541507,0.021581,0.050616,0.015032,0.015044,0.015147,...,0.090822,0.055321,0.065605,0.060629,0.055321,0.015563,0.586122,0.460040,0.145466,0.146851
5872,0.510204,0.0,0.733018,0.477393,0.531234,0.026825,0.050638,0.020801,0.020830,0.020856,...,0.104757,0.075127,0.076448,0.059640,0.075086,0.023283,0.646289,0.401075,0.184435,0.168492
5873,0.510204,0.0,0.764462,0.463237,0.521066,0.019968,0.040804,0.016605,0.015623,0.016604,...,0.050457,0.033031,0.040950,0.038759,0.032990,0.010292,0.628534,0.512392,0.140033,0.169155


In [138]:
from skimage.metrics import structural_similarity as ssim

feature_similarities = []

for column in original_data.columns:
    similarity_index, _ = ssim(
        original_data[column],
        synthetic_df[column],
        full=True,
        data_range=synthetic_df[column].max() - synthetic_df[column].min()
    )

    feature_similarities.append(similarity_index)

overall_similarity_index = np.mean(feature_similarities)

percentage_similarity = (overall_similarity_index + 1) * 50

print("Overall Percentage Similarity (SSI):", percentage_similarity)


Overall Percentage Similarity (SSI): 49.97288224019318


In [152]:
df_merge = pd.concat([original_data, synthetic_df])

In [153]:
X = df_merge.copy()
X = X.drop('total_UPDRS', axis=1)
y = df_merge['total_UPDRS']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=48)

In [154]:
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)
predictions = rf_model.predict(X_test)
mae_synthetic_merged = metrics.mean_absolute_error(y_test, predictions)
print(f'MAE original data: {mae}')
print(f'MAE original + synthetic data: {mae_synthetic_merged}')

MAE original data: 0.2104556212765951
MAE original + synthetic data: 0.002569680723152633


MAE from model trained on original data is more compared to MAE on the model trained on combined original and synthetic data.